In [69]:
import pandas as pd
import os
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [70]:
symbol = "^VIX"
data_dir = '../data/%s' % symbol
data_file = '%s_history.csv' %symbol;
file_path = os.path.join(data_dir, data_file)

In [71]:
data = pd.read_csv(file_path)

In [72]:
data.head(5)

,Date,Open,High,Low,Close,Volume
0,1990-01-02,17.24,17.24,17.24,17.24,0
1,1990-01-03,18.19,18.19,18.19,18.19,0
2,1990-01-04,19.22,19.22,19.22,19.22,0
3,1990-01-05,20.11,20.11,20.11,20.11,0
4,1990-01-08,20.26,20.26,20.26,20.26,0


In [73]:
price_pt_1 = 40
price_pt_2 = 20
fromDate = '2001-01-01'
toDate = '2021-03-01'
direction = 'UPTREND' if price_pt_1 < price_pt_2 else 'DOWNTREND'

In [74]:
analysis_data_window = data[(data['Date'] >= fromDate) & (data['Date'] <= toDate)]

In [75]:
analysis_data_window.head(5)

,Date,Open,High,Low,Close,Volume
2780,2001-01-02,27.93,30.40,27.92,29.99,0
2781,2001-01-03,29.63,29.96,25.99,26.60,0
2782,2001-01-04,26.35,27.19,26.13,26.97,0
2783,2001-01-05,27.81,29.04,27.21,28.67,0
2784,2001-01-08,29.98,30.80,29.72,29.84,0


In [76]:
analysis_data_window.tail(5)

,Date,Open,High,Low,Close,Volume
7845,2021-02-22,24.46,25.09,21.96,23.45,0
7846,2021-02-23,22.82,27.01,22.50,23.11,0
7847,2021-02-24,23.76,25.04,21.31,21.34,0
7848,2021-02-25,21.73,31.16,21.52,28.89,0
7849,2021-02-26,28.73,30.82,25.23,27.95,0


In [77]:
price_pt_1_all_occurences = analysis_data_window[analysis_data_window['High'] >= price_pt_1]

In [78]:
price_pt_1_all_occurences.head(5)

,Date,Open,High,Low,Close,Volume
2954,2001-09-17,43.20,44.33,39.77,41.76,0
2955,2001-09-18,41.07,42.17,38.87,38.87,0
2956,2001-09-19,37.88,44.26,37.50,40.56,0
2957,2001-09-20,41.40,43.76,41.17,43.74,0
2958,2001-09-21,48.93,49.35,42.66,42.66,0


In [79]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
end_date_obj = datetime.datetime.strptime(toDate, '%Y-%m-%d')

pricePt1Date = []
pricePt2Date = []
calendarDays = []
businessDays = []
pricePt1 = []
pricePt2 = []

while True:
    price_pt_1_date = price_pt_1_all_occurences.iloc[0]['Date']
    filtered_data_set = analysis_data_window[analysis_data_window['Date'] >= price_pt_1_date]
    if direction == 'UPTREND':
        price_pt_2_date = filtered_data_set[filtered_data_set['Close'] >= price_pt_2].iloc[0]['Date']
    else:
        price_pt_2_date = filtered_data_set[filtered_data_set['Close'] <= price_pt_2].iloc[0]['Date']
    
    price_pt_1_date_obj = datetime.datetime.strptime(price_pt_1_date, '%Y-%m-%d')
    price_pt_2_date_obj = datetime.datetime.strptime(price_pt_2_date, '%Y-%m-%d')
    
    total_cal_days_for_price_change =  (price_pt_2_date_obj - price_pt_1_date_obj).days
    total_business_days_for_price_change = len(pd.date_range(start = price_pt_1_date, end = price_pt_2_date, freq = us_bd))
    
    pricePt1Date.append(price_pt_1_date)
    pricePt2Date.append(price_pt_2_date)
    calendarDays.append(total_cal_days_for_price_change)
    businessDays.append(total_business_days_for_price_change)
    pricePt1.append(price_pt_1)
    pricePt2.append(price_pt_2)
    
    price_pt_1_all_occurences = price_pt_1_all_occurences[price_pt_1_all_occurences['Date'] > price_pt_2_date]
    
    if len(price_pt_1_all_occurences.index) == 0 :
        break
       
        
data = {'Price_Pt_1': pricePt1,
        'Price_Pt_1_Date': pricePt1Date, 
        'Price_Pt_2': pricePt2,
        'Price_Pt_2_Date': pricePt2Date,
        'Calendar Days': calendarDays,
        'Business Days': businessDays}
output_df = pd.DataFrame(data, columns= ['Price_Pt_1', 'Price_Pt_1_Date', 'Price_Pt_2', 'Price_Pt_2_Date', 'Calendar Days', 'Business Days'])

In [80]:
output_df.head(25)

,Price_Pt_1,Price_Pt_1_Date,Price_Pt_2,Price_Pt_2_Date,Calendar Days,Business Days
0,40,2001-09-17,20,2002-03-01,165,113
1,40,2002-07-22,20,2003-05-09,291,202
2,40,2008-09-18,20,2009-12-22,460,316
3,40,2010-05-06,20,2010-10-11,158,109
4,40,2011-08-08,20,2012-01-19,164,112
5,40,2015-08-24,20,2015-10-05,42,30
6,40,2018-02-06,20,2018-02-14,8,7
7,40,2020-02-28,20,2021-02-12,350,242
